In [6]:
import pandas as pd
from helpers import *
import os
from collections import Counter

In [7]:
NUM_OF_PUB = 284

In [8]:
OUTPUT = 'eval_output'
if not os.path.exists(OUTPUT):
    os.makedirs(OUTPUT)

In [9]:
# file path: update if needed
DECISION_KEYWORD_FILE = 'output/decision_df_keywords.csv'
ANNOTATION_FILE = 'data/Annotation_JCDL2024_YF.xlsx'
DECISION_ML_FILE = 'output/decision_df_tilo.csv'
OUTFILE_PATH = 'eval_output/performance_metrics.tsv'

## Evaluate decisions made by the keyword approach versus the silver standard

In [10]:
decision_keyword = pd.read_csv(DECISION_KEYWORD_FILE)
annotation = pd.read_excel(ANNOTATION_FILE, sheet_name='284')

In [11]:
decision_keyword.head()

,DOI,decision,rationale,stage
0,10.1002/chem.201601150,negligible risk,This paper is a review,1
1,10.1002/chem.202004790,negligible risk,This paper is a review,1
2,10.1002/mrc.4989,negligible risk,This paper is a review,1
3,10.1002/mrc.5234,negligible risk,This paper is a review,1
4,10.1002/mrc.5260,negligible risk,This paper is a review,1


In [12]:
decision_keyword = decision_keyword[['DOI','decision', 'stage']]
annotation = annotation[['DOI','Silver Standard']]
decision_keyword['DOI'] = decision_keyword['DOI'].apply(clean)
annotation['DOI'] = annotation['DOI'].apply(clean)

In [13]:
# eval_df = pd.merge(left=decision_keyword, right=annotation, on='DOI', how='left')
# eval_df.to_csv('eval_output/eval_keyword_silver.csv', index=False)

## Evaluate decisions made by the ML approach versus the silver standard

In [14]:
decision_df_tilo = pd.read_csv(DECISION_ML_FILE)

In [15]:
decision_df_tilo = decision_df_tilo[['DOI','decision', 'stage']]
decision_df_tilo['DOI'] = decision_df_tilo['DOI'].apply(clean)
decision_df_tilo.head()

,DOI,decision,stage
0,10.1002/anie.201706532,high risk,3
1,10.1002/anie.201708266,negligible risk,3
2,10.1002/anie.201810566,high risk,3
3,10.1002/anie.201902777,negligible risk,3
4,10.1002/anie.201916566,high risk,3


In [16]:
decision_df_tilo.loc[decision_df_tilo['decision'].isna()] = 'negligible risk'

In [17]:
# eval_df = pd.merge(left=decision_df_tilo, right=annotation, on='DOI', how='left')
# eval_df.to_csv('eval_output/eval_tilo_silver.csv', index=False)

## Merge data for all decisions

In [18]:
combined_df = pd.merge(left=annotation, right=decision_keyword, on='DOI', how='left').merge(right=decision_df_tilo.drop(columns=['stage']), on='DOI', how='left', suffixes=('_keyword', '_machine_learning'))

combined_df['decision_base'] = ['high risk']*combined_df.shape[0]

for idx in range(len(combined_df)):
    if combined_df['stage'][idx] == 1 or combined_df['stage'][idx] == 2:
        combined_df.loc[idx, 'decision_base'] = combined_df['decision_keyword'][idx]
        combined_df.loc[idx, 'decision_machine_learning'] = combined_df['decision_keyword'][idx]

### produced results shown in Table 3 of the manuscript

In [19]:
# conversion
combined_df['decision_keyword_conv'] = [conversion(silver, decision) for silver, decision in zip(combined_df['Silver Standard'], combined_df['decision_keyword'])]

combined_df['decision_machine_learning_conv'] = [conversion(silver, decision) for silver, decision in zip(combined_df['Silver Standard'], combined_df['decision_machine_learning'])]

combined_df['decision_base_conv'] = [conversion(silver, decision) for silver, decision in zip(combined_df['Silver Standard'], combined_df['decision_base'])]

In [20]:
with open(OUTFILE_PATH, 'w') as f:
    f.write('Approaches\tFalse Negative Rate (%)\tFalse Positive Rate (%)\n')
    
computing_performance(combined_df['decision_base_conv'], 'Base approach (Stages 1 & 2)', OUTFILE_PATH)
computing_performance(combined_df['decision_keyword_conv'], 'Approach-KW (Stages 1, 2, & 3)', OUTFILE_PATH)
computing_performance(combined_df.loc[combined_df['stage'] == 3, 'decision_keyword_conv'], 'The keyword-based decision tree module (Q4 and Q5) in Approach-KW (Stage 3)', OUTFILE_PATH)
computing_performance(combined_df['decision_machine_learning_conv'], 'Approach-ML (Stages 1, 2, & 3)', OUTFILE_PATH)
computing_performance(combined_df.loc[combined_df['stage'] == 3, 'decision_machine_learning_conv'], 'The machine learning-based module in Approach-ML (Stage 3)', OUTFILE_PATH)
